<a href="https://colab.research.google.com/github/linyuehzzz/census_privacy/blob/main/franklin_microsim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


#### **Select variables**
Constraint variables:
- sex (SEX)
- race (RAC1P)
- age band (AGEP)  

Target variables: 
- disability (DIS)
- married, spouse present/spouse absent (MSP)
- mobility status (MIG)
- military service (MIL)
- educational attainment (SCHL)
- recoded field of degree - first entry (FOD1P)
- health insurance coverage (HICOV)
- private health insurance coverage recode (PRIVCOV)
- public health coverage recode (PUBCOV)
- total person's income (PINCP)
- income-to-poverty ratio recode (POVPIP)
- class of worker (COW)
- employment status recode (ESR)
- Standard Occupational Classification (SOC) codes for 2018 and later based on 2018 SOC codes (SOCP)
- travel time to work (JWMNP)
- vehicle occupancy (JWRIP)
- means of transportation to work (JWTRNS)

In [ ]:
%cd "/content/gdrive/My Drive/Colab Notebooks/census_privacy"
import pandas as pd

filename_pums = 'franklin/pums/franklin_pums10.csv'
data_pums = pd.read_csv(filename_pums)
pums_subset = data_pums[["PUMAID", "SEX", "RAC1P", "AGEP", "DIS", "MSP", "MIG", "MIL", "SCHL", 
"FOD1P", "HICOV", "PRIVCOV", "PUBCOV", "PINCP", "POVPIP", "COW", "ESR", "SOCP", "JWMNP", "JWRIP", "JWTRNS"]]
pums_subset.to_csv("franklin/pums/franklin_pums10v1.csv", index=False)
pums_subset

/content/gdrive/My Drive/Colab Notebooks/census_privacy


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,PUMAID,SEX,RAC1P,AGEP,DIS,MSP,MIG,MIL,SCHL,FOD1P,HICOV,PRIVCOV,PUBCOV,PINCP,POVPIP,COW,ESR,SOCP,JWMNP,JWRIP,JWTRNS
0,3904103,2,1,48,2,1.0,1.0,4.0,21.0,6203.0,1,1,2,15000.0,501.0,1.0,1.0,132052,20.0,1.0,1.0
1,3904103,1,1,48,2,1.0,1.0,4.0,21.0,6207.0,1,1,2,155000.0,501.0,1.0,1.0,1191XX,15.0,1.0,1.0
2,3904103,1,1,18,2,6.0,1.0,4.0,14.0,NaN,1,1,2,0.0,501.0,NaN,6.0,NaN,NaN,NaN,NaN
3,3904103,1,1,16,2,6.0,1.0,NaN,12.0,NaN,1,1,2,0.0,501.0,NaN,6.0,NaN,NaN,NaN,NaN
4,3904103,1,1,14,2,NaN,1.0,NaN,10.0,NaN,1,1,2,NaN,501.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52846,3904102,2,1,63,2,4.0,1.0,4.0,19.0,NaN,1,1,2,37000.0,281.0,1.0,1.0,311131,20.0,1.0,1.0
52847,3904102,2,1,36,2,4.0,3.0,4.0,19.0,NaN,1,1,2,25800.0,197.0,1.0,1.0,439061,10.0,1.0,1.0
52848,3904106,1,1,94,1,6.0,1.0,2.0,19.0,NaN,1,1,1,152100.0,501.0,NaN,6.0,NaN,NaN,NaN,NaN
52849,3904105,2,1,46,2,6.0,1.0,4.0,21.0,5502.0,1,1,2,83000.0,501.0,4.0,1.0,291141,15.0,1.0,1.0


Prepare individual data (*ind*)

In [ ]:
%cd "/content/gdrive/My Drive/Colab Notebooks/census_privacy"
import pandas as pd
from lib.target_constraints import *

filename_pums = 'franklin/pums/psam_p39.csv'
data_pums = pd.read_csv(filename_pums)
pums_subset = data_pums[["RAC1P", "RAC3P", "AGEP", "HISP", "RELSHIPP", "SEX"]]
pums_subset.to_csv("franklin/pums/psam_p39v2.csv", index=False)

pums_subset

/content/gdrive/My Drive/Colab Notebooks/census_privacy


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,RAC1P,RAC3P,AGEP,HISP,RELSHIPP,SEX
0,1,1,34,1,20,1
1,1,1,41,1,21,2
2,1,1,9,1,25,1
3,1,1,21,1,27,1
4,1,1,36,1,20,1
...,...,...,...,...,...,...
593061,1,1,58,1,22,1
593062,1,1,17,1,26,2
593063,1,1,53,1,20,2
593064,1,1,55,1,21,1


In [ ]:
%cd "/content/gdrive/My Drive/Colab Notebooks/census_privacy"
import pandas as pd
from lib.target_constraints import *

# filename_pums = 'franklin/pums/psam_p39v2.csv'
filename_pums = 'franklin/pums/psam_p39v2.csv'
data_pums = pd.read_csv(filename_pums)
new_cols = ["RACES", "RACEL", "AGES", "AGEL", "HISP2", "RELSHIPP2"]
data_pums = data_pums.join(pd.DataFrame(0, columns=new_cols, index=data_pums.index))

for index, row in data_pums.iterrows():
    # race (short, 7 cells)
    data_pums.iloc[index]["RACES"] = convert_race1(row["RAC1P"])
    # race (long, 63 cells)
    data_pums.iloc[index]["RACEL"] = convert_race2(row["RAC3P"])
    # age (short, 3 cells)
    data_pums.iloc[index]["AGES"] = convert_age1(row["AGEP"])
    # age (long, 23 cells)
    data_pums.iloc[index]["AGEL"] = convert_age2(row["AGEP"])
    # hispanic (2 cells)
    data_pums.iloc[index]["HISP2"] = convert_hisp1(row["HISP"])
    # unit type (3 cells)
    data_pums.iloc[index]["RELSHIPP2"] = convert_hhgq1(row["RELSHIPP"])
data_pums

/content/gdrive/My Drive/Colab Notebooks/census_privacy


,RAC1P,RAC3P,AGEP,HISP,RELSHIPP,SEX,RACES,RACEL,AGES,AGEL,HISP2,RELSHIPP2
0,1,1,34,1,20,1,1,1,2,10,1,1
1,1,1,41,1,21,2,1,1,2,12,1,1
2,1,1,9,1,25,1,1,1,1,2,1,1
3,1,1,21,1,27,1,1,1,2,7,1,1
4,1,1,36,1,20,1,1,1,2,11,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
593061,1,1,58,1,22,1,1,1,2,15,1,1
593062,1,1,17,1,26,2,1,1,1,4,1,1
593063,1,1,53,1,20,2,1,1,2,14,1,1
593064,1,1,55,1,21,1,1,1,2,15,1,1


In [ ]:
## create ind file for each PUMA
%cd "/content/gdrive/My Drive/Colab Notebooks/census_privacy"
import pandas as pd

## initialize ind
p5_col = ["P00500%s" % '{number:0{width}d}'.format(width=2, number=x) for x in range(1, 18)]
p8_col = ["P00800%s" % '{number:0{width}d}'.format(width=2, number=x) for x in range(1, 72)]
p9_col = ["P00900%s" % '{number:0{width}d}'.format(width=2, number=x) for x in range(1, 74)]
p12A_col = ["P012A0%s" % '{number:0{width}d}'.format(width=2, number=x) for x in range(1, 50)]
p12B_col = ["P012B0%s" % '{number:0{width}d}'.format(width=2, number=x) for x in range(1, 50)]
p12C_col = ["P012C0%s" % '{number:0{width}d}'.format(width=2, number=x) for x in range(1, 50)]
p12D_col = ["P012D0%s" % '{number:0{width}d}'.format(width=2, number=x) for x in range(1, 50)]
p12E_col = ["P012E0%s" % '{number:0{width}d}'.format(width=2, number=x) for x in range(1, 50)]
p12F_col = ["P012F0%s" % '{number:0{width}d}'.format(width=2, number=x) for x in range(1, 50)]
p12G_col = ["P012G0%s" % '{number:0{width}d}'.format(width=2, number=x) for x in range(1, 50)]
p43_col = ["P0430004", "P0430009", "P0430014", "P0430019", "P0430024", "P0430029", "P0430035", "P0430040", "P0430045", "P0430050",
           "P0430055", "P0430060"]
# column_names = ["PUMAID"]
column_names = []
column_names.extend(p5_col + p8_col + p9_col + p12A_col + p12B_col + p12C_col + p12D_col + p12E_col + p12F_col + p12G_col + p43_col)
unwanted_col = ["P0050001", "P0050002", "P0050010", "P0080001", "P0080002", "P0080009", "P0080010", "P0080026", "P0080047", "P0080063", 
                "P0080070", "P0090001", "P0090002", "P0090003", "P0090004", "P0090011", "P0090012", "P0090028", "P0090049", "P0090065", 
                "P0090072", "P012A001", "P012A002", "P012A026", "P012B001", "P012B002", "P012B026", "P012C001", "P012C002", "P012C026",
                "P012D001", "P012D002", "P012D026", "P012E001", "P012E002", "P012E026", "P012F001", "P012F002", "P012F026", "P012G001", 
                "P012G002", "P012G026"]
column_names = [ele for ele in column_names if ele not in unwanted_col]
ind = pd.DataFrame(0, columns = column_names, index=data_pums.index)
# ind["PUMAID"] = data_pums["PUMAID"]

## re-categorize ind
for index, row in data_pums.iterrows():
    ## P5: HISPANIC OR LATINO ORIGIN BY RACE (2 * 7)
    if row["HISP2"] == 1 and row["RACES"] >= 1 and row["RACES"] <= 7:
        ind.iloc[index]["P00500%s" % '{number:0{width}d}'.format(width=2, number=(row["RACES"] + 2))] = 1
    elif row["HISP2"] == 2 and row["RACES"] >= 1 and row["RACES"] <= 7:
        ind.iloc[index]["P00500%s" % '{number:0{width}d}'.format(width=2, number=(row["RACES"] + 10))] = 1
    
    ## P8: RACE (63)
    if row["RACEL"] >= 1 and row["RACEL"] <= 6:
        ind.iloc[index]["P00800%s" % '{number:0{width}d}'.format(width=2, number=(row["RACEL"] + 2))] = 1
    elif row["RACEL"] >= 7 and row["RACEL"] <= 21:
        ind.iloc[index]["P00800%s" % '{number:0{width}d}'.format(width=2, number=(row["RACEL"] + 4))] = 1
    elif row["RACEL"] >= 22 and row["RACEL"] <= 41:
        ind.iloc[index]["P00800%s" % '{number:0{width}d}'.format(width=2, number=(row["RACEL"] + 5))] = 1
    elif row["RACEL"] >= 42 and row["RACEL"] <= 56:
        ind.iloc[index]["P00800%s" % '{number:0{width}d}'.format(width=2, number=(row["RACEL"] + 6))] = 1
    elif row["RACEL"] >= 57 and row["RACEL"] <= 62:
        ind.iloc[index]["P00800%s" % '{number:0{width}d}'.format(width=2, number=(row["RACEL"] + 7))] = 1
    elif row["RACEL"] == 63:
        ind.iloc[index]["P00800%s" % '{number:0{width}d}'.format(width=2, number=(row["RACEL"] + 8))] = 1

    ## P9: HISPANIC OR LATINO, AND NOT HISPANIC OR LATINO BY RACE (1 * 63)
    if row["HISP2"] == 1 and row["RACEL"] >= 1 and row["RACEL"] <= 6:
        ind.iloc[index]["P00900%s" % '{number:0{width}d}'.format(width=2, number=(row["RACEL"] + 4))] = 1
    elif row["HISP2"] == 1 and row["RACEL"] >= 7 and row["RACEL"] <= 21:
        ind.iloc[index]["P00900%s" % '{number:0{width}d}'.format(width=2, number=(row["RACEL"] + 6))] = 1
    elif row["HISP2"] == 1 and row["RACEL"] >= 22 and row["RACEL"] <= 41:
        ind.iloc[index]["P00900%s" % '{number:0{width}d}'.format(width=2, number=(row["RACEL"] + 7))] = 1
    elif row["HISP2"] == 1 and row["RACEL"] >= 42 and row["RACEL"] <= 56:
        ind.iloc[index]["P00900%s" % '{number:0{width}d}'.format(width=2, number=(row["RACEL"] + 8))] = 1
    elif row["HISP2"] == 1 and row["RACEL"] >= 57 and row["RACEL"] <= 62:
        ind.iloc[index]["P00900%s" % '{number:0{width}d}'.format(width=2, number=(row["RACEL"] + 9))] = 1
    elif row["HISP2"] == 1 and row["RACEL"] == 63:
        ind.iloc[index]["P00900%s" % '{number:0{width}d}'.format(width=2, number=(row["RACEL"] + 10))] = 1

    ## P12A: SEX BY AGE (WHITE ALONE)
    if row["RACES"] == 1 and row["SEX"] == 1 and row["AGEL"] >= 1 and row["AGEL"] <= 23:
        ind.iloc[index]["P012A0%s" % '{number:0{width}d}'.format(width=2, number=(row["AGEL"] + 2))] = 1
    elif row["RACES"] == 1 and row["SEX"] == 2 and row["AGEL"] >= 1 and row["RACES"] <= 23:
        ind.iloc[index]["P012A0%s" % '{number:0{width}d}'.format(width=2, number=(row["AGEL"] + 26))] = 1
    
    ## P12B: SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)
    if row["RACES"] == 2 and row["SEX"] == 1 and row["AGEL"] >= 1 and row["AGEL"] <= 23:
        ind.iloc[index]["P012B0%s" % '{number:0{width}d}'.format(width=2, number=(row["AGEL"] + 2))] = 1
    elif row["RACES"] == 2 and row["SEX"] == 2 and row["AGEL"] >= 1 and row["RACES"] <= 23:
        ind.iloc[index]["P012B0%s" % '{number:0{width}d}'.format(width=2, number=(row["AGEL"] + 26))] = 1
    
    ## P12C: SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ALONE)
    if row["RACES"] == 3 and row["SEX"] == 1 and row["AGEL"] >= 1 and row["AGEL"] <= 23:
        ind.iloc[index]["P012C0%s" % '{number:0{width}d}'.format(width=2, number=(row["AGEL"] + 2))] = 1
    elif row["RACES"] == 3 and row["SEX"] == 2 and row["AGEL"] >= 1 and row["RACES"] <= 23:
        ind.iloc[index]["P012C0%s" % '{number:0{width}d}'.format(width=2, number=(row["AGEL"] + 26))] = 1    

    ## P12D: SEX BY AGE (ASIAN ALONE)
    if row["RACES"] == 4 and row["SEX"] == 1 and row["AGEL"] >= 1 and row["AGEL"] <= 23:
        ind.iloc[index]["P012D0%s" % '{number:0{width}d}'.format(width=2, number=(row["AGEL"] + 2))] = 1
    elif row["RACES"] == 4 and row["SEX"] == 2 and row["AGEL"] >= 1 and row["RACES"] <= 23:
        ind.iloc[index]["P012D0%s" % '{number:0{width}d}'.format(width=2, number=(row["AGEL"] + 26))] = 1   

    ## P12E: SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)
    if row["RACES"] == 5 and row["SEX"] == 1 and row["AGEL"] >= 1 and row["AGEL"] <= 23:
        ind.iloc[index]["P012E0%s" % '{number:0{width}d}'.format(width=2, number=(row["AGEL"] + 2))] = 1
    elif row["RACES"] == 5 and row["SEX"] == 2 and row["AGEL"] >= 1 and row["RACES"] <= 23:
        ind.iloc[index]["P012E0%s" % '{number:0{width}d}'.format(width=2, number=(row["AGEL"] + 26))] = 1  

    ## P12F: SEX BY AGE (SOME OTHER RACE ALONE)
    if row["RACES"] == 6 and row["SEX"] == 1 and row["AGEL"] >= 1 and row["AGEL"] <= 23:
        ind.iloc[index]["P012F0%s" % '{number:0{width}d}'.format(width=2, number=(row["AGEL"] + 2))] = 1
    elif row["RACES"] == 6 and row["SEX"] == 2 and row["AGEL"] >= 1 and row["RACES"] <= 23:
        ind.iloc[index]["P012F0%s" % '{number:0{width}d}'.format(width=2, number=(row["AGEL"] + 26))] = 1  

    ## P12G: SEX BY AGE (TWO OR MORE RACES)
    if row["RACES"] == 7 and row["SEX"] == 1 and row["AGEL"] >= 1 and row["AGEL"] <= 23:
        ind.iloc[index]["P012G0%s" % '{number:0{width}d}'.format(width=2, number=(row["AGEL"] + 2))] = 1
    elif row["RACES"] == 7 and row["SEX"] == 2 and row["AGEL"] >= 1 and row["RACES"] <= 23:
        ind.iloc[index]["P012G0%s" % '{number:0{width}d}'.format(width=2, number=(row["AGEL"] + 26))] = 1  
    
    ## P43: GROUP QUARTERS POPULATION BY SEX BY AGE BY GROUP QUARTERS TYPE
    if row["SEX"] == 1 and row["AGES"] == 1 and row["RELSHIPP2"] == 2:
        ind.iloc[index]["P0430004"] = 1
    elif row["SEX"] == 1 and row["AGES"] == 1 and row["RELSHIPP2"] == 3:
        ind.iloc[index]["P0430009"] = 1
    elif row["SEX"] == 1 and row["AGES"] == 2 and row["RELSHIPP2"] == 2:
        ind.iloc[index]["P0430014"] = 1
    elif row["SEX"] == 1 and row["AGES"] == 2 and row["RELSHIPP2"] == 3:
        ind.iloc[index]["P0430019"] = 1    
    elif row["SEX"] == 1 and row["AGES"] == 3 and row["RELSHIPP2"] == 2:
        ind.iloc[index]["P0430024"] = 1
    elif row["SEX"] == 1 and row["AGES"] == 3 and row["RELSHIPP2"] == 3:
        ind.iloc[index]["P0430029"] = 1
    elif row["SEX"] == 2 and row["AGES"] == 1 and row["RELSHIPP2"] == 2:
        ind.iloc[index]["P0430035"] = 1
    elif row["SEX"] == 2 and row["AGES"] == 1 and row["RELSHIPP2"] == 3:
        ind.iloc[index]["P0430040"] = 1
    elif row["SEX"] == 2 and row["AGES"] == 2 and row["RELSHIPP2"] == 2:
        ind.iloc[index]["P0430045"] = 1
    elif row["SEX"] == 2 and row["AGES"] == 2 and row["RELSHIPP2"] == 3:
        ind.iloc[index]["P0430050"] = 1    
    elif row["SEX"] == 2 and row["AGES"] == 3 and row["RELSHIPP2"] == 2:
        ind.iloc[index]["P0430055"] = 1
    elif row["SEX"] == 2 and row["AGES"] == 3 and row["RELSHIPP2"] == 3:
        ind.iloc[index]["P0430060"] = 1    

# for puma in ind["PUMAID"].unique():
#     ind_subset = ind[ind["PUMAID"] == puma]
#     ind_subset = ind_subset.drop(columns=["PUMAID"])
#     ind_subset.to_csv("franklin/microdata/ipf/franklin_ind_" + str(puma) + ".csv", index=False)
#     print(puma, ind_subset.shape)

# ind_all = ind.drop(columns=["PUMAID"])
# ind_all.to_csv("franklin/microdata/ipf/franklin_ind.csv", index=False)
# ind_all

ind.to_csv("franklin/microdata/ipf/ohio_ind.csv", index=False)
ind

/content/gdrive/My Drive/Colab Notebooks/census_privacy


,P0050003,P0050004,P0050005,P0050006,P0050007,P0050008,P0050009,P0050011,P0050012,P0050013,P0050014,P0050015,P0050016,P0050017,P0080003,P0080004,P0080005,P0080006,P0080007,P0080008,P0080011,P0080012,P0080013,P0080014,P0080015,P0080016,P0080017,P0080018,P0080019,P0080020,P0080021,P0080022,P0080023,P0080024,P0080025,P0080027,P0080028,P0080029,P0080030,P0080031,...,P012G021,P012G022,P012G023,P012G024,P012G025,P012G027,P012G028,P012G029,P012G030,P012G031,P012G032,P012G033,P012G034,P012G035,P012G036,P012G037,P012G038,P012G039,P012G040,P012G041,P012G042,P012G043,P012G044,P012G045,P012G046,P012G047,P012G048,P012G049,P0430004,P0430009,P0430014,P0430019,P0430024,P0430029,P0430035,P0430040,P0430045,P0430050,P0430055,P0430060
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593061,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
593062,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
593063,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
593064,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### **Microsimulation**

First, we read the microdata (1163414 records) with only the constraint variables (race/sex/age band). This data set is an enumeration of the population in Franklin County, which is partitioned into 22,826 blocks.

In [ ]:
%cd "/content/gdrive/My Drive/Colab Notebooks/census_privacy"
import pandas as pd

filename_c = 'franklin/microdata/franklin_people_p12.csv'
data_c = pd.read_csv(filename_c)
data_c

/content/gdrive/My Drive/Colab Notebooks/census_privacy


,GEOID10,race,sex,age,age_v,PUMA
0,390490001101001,1,1,9,28,3904102
1,390490001101001,7,2,9,27,3904102
2,390490001101002,1,2,1,0,3904102
3,390490001101002,1,2,1,3,3904102
4,390490001101002,1,2,2,7,3904102
...,...,...,...,...,...,...
1163409,390490107001020,1,2,21,75,3904102
1163410,390490107001020,1,1,23,95,3904102
1163411,390490107001020,1,2,23,97,3904102
1163412,390499800001027,1,1,20,70,3904106


We then read the PUMS that covers approximately 5% of the population (52851 records) in Franklin County. This data set is at the PUA level, which partitions the Franklin County into 11 PUMAs. Compared to the previous data set, it has more other variables of interest.

In [ ]:
%cd "/content/gdrive/My Drive/Colab Notebooks/census_privacy"
filename_pums = 'franklin/pums/franklin_pums10v1.csv'
data_pums = pd.read_csv(filename_pums)
data_pums

/content/gdrive/My Drive/Colab Notebooks/census_privacy


,PUMAID,SEX,RAC1P,AGEP,DIS,MSP,MIG,MIL,SCHL,FOD1P,HICOV,PRIVCOV,PUBCOV,PINCP,POVPIP,COW,ESR,SOCP,JWMNP,JWRIP,JWTRNS
0,3904103,2,1,48,2,1.0,1.0,4.0,21.0,6203.0,1,1,2,15000.0,501.0,1.0,1.0,132052,20.0,1.0,1.0
1,3904103,1,1,48,2,1.0,1.0,4.0,21.0,6207.0,1,1,2,155000.0,501.0,1.0,1.0,1191XX,15.0,1.0,1.0
2,3904103,1,1,18,2,6.0,1.0,4.0,14.0,NaN,1,1,2,0.0,501.0,NaN,6.0,NaN,NaN,NaN,NaN
3,3904103,1,1,16,2,6.0,1.0,NaN,12.0,NaN,1,1,2,0.0,501.0,NaN,6.0,NaN,NaN,NaN,NaN
4,3904103,1,1,14,2,NaN,1.0,NaN,10.0,NaN,1,1,2,NaN,501.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52846,3904102,2,1,63,2,4.0,1.0,4.0,19.0,NaN,1,1,2,37000.0,281.0,1.0,1.0,311131,20.0,1.0,1.0
52847,3904102,2,1,36,2,4.0,3.0,4.0,19.0,NaN,1,1,2,25800.0,197.0,1.0,1.0,439061,10.0,1.0,1.0
52848,3904106,1,1,94,1,6.0,1.0,2.0,19.0,NaN,1,1,1,152100.0,501.0,NaN,6.0,NaN,NaN,NaN,NaN
52849,3904105,2,1,46,2,6.0,1.0,4.0,21.0,5502.0,1,1,2,83000.0,501.0,4.0,1.0,291141,15.0,1.0,1.0


#### **Traditional method**
- Reweighting existing survey data (not creating new data)
- Treating individuals with the same constraint variables as identical

For each PUMA, we randomly sample the records in PUMS that match the constraint variables in the enumeration data set.

In [ ]:
%cd "/content/gdrive/My Drive/Colab Notebooks/census_privacy"
import numpy as np 
import pandas as pd
import csv
from lib.target_constraints import convert_race, convert_sex, convert_age

output = 'franklin/microdata/franklin_peoplev0.csv'
a = ["GEOID10"]
head = list(a)
head.extend(data_pums.columns.tolist())

with open(output, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(head)
    for index, row in data_c.iterrows():
        # constraint variables
        puma = row["PUMA"]
        race = convert_race(row["race"])
        sex = convert_sex(row["sex"])
        age_band = convert_age(row["age"])

        # match
        sub_pums = data_pums[(data_pums["PUMAID"] == puma) & (data_pums["SEX"] == sex) & (data_pums["RAC1P"] == race) 
        & (data_pums["AGEP"] >= age_band[0]) & (data_pums["AGEP"] <= age_band[1])]

        if len(sub_pums) > 0:
            sub_pums = sub_pums.sample(n=1)
            GEOID10 = [row["GEOID10"]]
            var = GEOID10 + sub_pums.values.tolist()[0]
            writer.writerow(var)

The traditional method is designed to "weight" each individual in the sample survey to match the aggregated statistics of the constraint variables in the census. However, one limitation of such a method is that if the survey does not cover some minorities (e.g., old man that is still working) in the population, they can not appear in the final microdata product.

Here, we calculate how many people are not able to find a match in the survey and thus are excluded in the final product.

In [ ]:
%cd "/content/gdrive/My Drive/Colab Notebooks/census_privacy"
filename_c = 'franklin/microdata/franklin_people_p12.csv'
data_c = pd.read_csv(filename_c)

filename_o = 'franklin/microdata/franklin_peoplev1.csv'
data_o = pd.read_csv(filename_o)

m_cnt = len(data_c) - len(data_o)

print(str(m_cnt) + " records do not have matched survey data.")

10810 records do not have matched survey data.


#### **Statistical learning**


The `sdv` library is used here. We load the PUMS data as the training data. We will approaximate the joint probability distribution of all the variables based on this data set, from which we will sample our synthetic individuals.

In [ ]:
!pip install sdv

#####**Define custom constraints**

Define some custom constraints based on reject sampling using the `is_valid` function. Constraints to be considered based on the 2015-2019 ACS PUMS Data Dictionary:
- People aged below 15 should not be married (MSP = b).
- People aged below 1 should not move a year ago (MIG = b).
- People aged below 17 should not serve in military (MIL = b).
- People aged below 3 do not have educational attainment (SCHL = b).
- People with less than bachelor's degree (SCHL = 01 to 20) do not have field of degree (FOD1P = bbbb).
- People aged below 15 do not have income (PINCP = bbbbbbb).
- People aged below 16 should not be working (COW = ESR = SOCP =b).
- People aged below 16 or unemployed/not in labor force (COW = 9 or ESR = 3/6 or SOCP = 999920) do not travel to work (JWMNP = JWTRNS = b).
- People not working (COW = 9 or ESR = 3/6 or SOCP = 999920) or not driving to work (JWTRNS != 01) do not occupy a car (JWRIP = bb).
- Any code not in the dictionary.

Pack everything together in `CustomConstraint`.

In [ ]:
from sdv.constraints import CustomConstraint
from lib.custom_constraints import is_valid

constraint = CustomConstraint(is_valid=is_valid)

##### **Fit models**

Gaussian Copula, CTGAN, and TVAE

Fit the `GaussainCopula` model.

In [ ]:
%cd "/content/gdrive/My Drive/Colab Notebooks/census_privacy"
from sdv.tabular import GaussianCopula

for puma in data_pums['PUMAID'].unique():
    print(puma)
    sub_pums = data_pums[data_pums['PUMAID'] == puma]
    model_gau = GaussianCopula(constraints=[constraint])
    model_gau.fit(sub_pums)
    model_gau.save('models/model_gau_' + str(puma) + '.pkl')

Fit the `CTGAN` model.

In [ ]:
%cd "/content/gdrive/My Drive/Colab Notebooks/census_privacy"
from sdv.tabular import CTGAN

for puma in data_pums['PUMAID'].unique():
    print(puma)
    sub_pums = data_pums[data_pums['PUMAID'] == puma]
    model_ctgan = CTGAN(constraints=[constraint])
    model_ctgan.fit(sub_pums)
    model_ctgan.save('models/model_ctgan_' + str(puma) + '.pkl')

Fit the `TVAE` model.

In [ ]:
%cd "/content/gdrive/My Drive/Colab Notebooks/census_privacy"
from sdv.tabular import TVAE

for puma in data_pums['PUMAID'].unique():
    print(puma)
    sub_pums = data_pums[data_pums['PUMAID'] == puma]
    model_tvae = TVAE(constraints=[constraint])
    model_tvae.fit(sub_pums)
    model_tvae.save('models/model_tvae_' + str(puma) + '.pkl')

##### **Conditional sampling**

Now that we have the model that allows us to approximate the joint distribution of the variables, we can create a sample from that distribution that matches our constraint variables (sex/race/age) in the enumeration data set. We use conditional sampling to generate the synthetic population.

In [ ]:
%cd "/content/gdrive/My Drive/Colab Notebooks/census_privacy"
from sdv.tabular import GaussianCopula
import numpy as np
from numpy import nan 
import pandas as pd
import csv
from lib.target_constraints import convert_race, convert_sex, convert_age
from lib.custom_constraints import is_valid

output = 'franklin/microdata/franklin_peoplev1.csv'
a = ["GEOID10"]
head = list(a)
head.extend(data_pums.columns.tolist())

N = 1000

with open(output, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(head)
    for index, row in data_c[:10].iterrows():
        # load model
        puma = row["PUMA"]
        model_gau = GaussianCopula.load('models/model_gau_' + str(puma) + '.pkl')

        # constraint variables
        race = convert_race(row["race"])
        sex = convert_sex(row["sex"])
        age_band = convert_age(row["age"])

        # conditions = {
        #     'RAC1P': race,
        #     'SEX': sex,
        #     'AGEP': age
        # }

        # rejection sampling
        sub_pums = []
        flag = 0
        while True:
            # sex, race, and age constraints
            # sub_pums = model_gau.sample(1, conditions=conditions)
            sub_pums = model_gau.sample(N)
            sub_pums = sub_pums[(data_pums["SEX"] == sex) & (sub_pums["RAC1P"] == race) 
            & (sub_pums["AGEP"] >= age_band[0]) & (sub_pums["AGEP"] <= age_band[1])]
            flag += 1
            if len(sub_pums) > 0:
                break
            if flag > 50:
                print("Consider increasing sampling size.")

        sub_pums = sub_pums.sample(n=1)
        sub_pums["PUMAID"] = puma
        print(sub_pums)
        GEOID10 = [row["GEOID10"]]
        var = GEOID10 + sub_pums.values.tolist()[0]
        writer.writerow(var)